## Práctica 0: El Hash Dorado

In [2]:
import random as rd
MOD = 1000000

def es_dorado(coord):
    return hash(coord) % MOD == 0


Nota: coord debe ser una tupla de dos enteros


Nos movemos coordenadas de 1 en 1

In [3]:
def paso_aleatorio(x,y):
    dx, dy = rd.choice([(1,0), (-1,0), (0,1), (0,-1)])
    return x + dx, y + dy

Creamos la funcion lupa, para buscar el hash dorado

In [4]:
def lupa(origen):
    x0, y0 = origen
    x,y = x0, y0
    n=0
    while True:
        x,y = paso_aleatorio(x,y)
        n+=1
        if es_dorado((x,y)):
            print(n)
            return(x,y)

Creamos 8 coordenadas separadas

In [5]:
S = 100_000
starts = [(i*S, 0) for i in range(8)]

Ejecutamos 8 lupas de forma secuencial.

In [6]:
import time

resultados = []

t0 = time.perf_counter()

for start in starts:
    p = lupa(start)
    resultados.append(p)

t1 = time.perf_counter()

print("tiempo total:", t1 - t0)
print("resultados:", resultados)


8947024
2891783
19334889
7485506
6842171
520466
3687666
722485
tiempo total: 30.39637800003402
resultados: [(402, -584), (99091, 3894), (203350, -2623), (299983, -1333), (397635, 1158), (500013, 435), (602336, 510), (701086, -251)]


8 lupas concurrentes con hilos (Threading)

In [7]:
import threading
import time

def worker(idx, start, out):
    out[idx] = lupa(start)

resultados = [None] * len(starts)
threads = []

t0 = time.perf_counter()

for i, start in enumerate(starts):
    th = threading.Thread(target=worker, args=(i, start, resultados))
    th.start()
    threads.append(th)

for th in threads:
    th.join()

t1 = time.perf_counter()
print("tiempo total (threads):", t1 - t0)
print("resultados:", resultados)


36144
79564
1034046
1322455
1970924
3595800
4040666
10102597
tiempo total (threads): 13.214848499977961
resultados: [(1494, 1681), (101480, -780), (201019, -1073), (301831, 395), (400125, -169), (499759, -125), (599552, 142), (700147, -258)]


In [8]:
import threading
import time

def lupa_con_stop(origen, stop_event):
    x0, y0 = origen
    x, y = x0, y0
    n = 0
    while not stop_event.is_set():
        x, y = paso_aleatorio(x, y)
        n += 1
        if es_dorado((x, y)):
            print(n)
            return (x, y)
    return None


In [9]:
def concurso_threads(starts):
    stop = threading.Event()
    ganador = {"start": None, "coord": None}
    lock = threading.Lock()

    def worker(start):
        coord = lupa_con_stop(start, stop)
        if coord is not None:
            with lock:
                if not stop.is_set():
                    ganador["start"] = start
                    ganador["coord"] = coord
                    stop.set()

    threads = []
    t0 = time.perf_counter()

    for s in starts:
        th = threading.Thread(target=worker, args=(s,))
        th.start()
        threads.append(th)

    for th in threads:
        th.join()

    t1 = time.perf_counter()
    return ganador, (t1 - t0)

ganador, tiempo = concurso_threads(starts)
print("ganador:", ganador)
print("tiempo concurso (threads):", tiempo)


93030
ganador: {'start': (400000, 0), 'coord': (400125, -169)}
tiempo concurso (threads): 0.6155189168639481


8 lupas concurrentes con procesos (Multiprocessing)

In [16]:
import multiprocessing as mp
import time

t0 = time.perf_counter()
with mp.Pool(processes=8) as pool:
    resultados_proc = pool.map(lupa, starts)
t1 = time.perf_counter()

print("tiempo total (procesos):", t1 - t0)
print("resultados:", resultados_proc)

Process SpawnPoolWorker-37:
Process SpawnPoolWorker-38:
Process SpawnPoolWorker-40:
Process SpawnPoolWorker-39:
Process SpawnPoolWorker-35:
Process SpawnPoolWorker-33:
Process SpawnPoolWorker-36:
Process SpawnPoolWorker-34:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/anaconda3/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^

KeyboardInterrupt: 

Concurso con procesos

In [ ]:
import multiprocessing as mp
import time

def lupa_proceso_concurso(origen, stop_event, q):
    x0, y0 = origen
    x, y = x0, y0
    n = 0

    while not stop_event.is_set():
        x, y = paso_aleatorio(x, y)
        n += 1
        if es_dorado((x, y)):
            q.put((origen, (x, y), n))
            stop_event.set()
            return

def concurso_procesos(starts):
    stop = mp.Event()
    q = mp.Queue()
    procs = []

    t0 = time.perf_counter()

    for s in starts:
        p = mp.Process(target=lupa_proceso_concurso, args=(s, stop, q))
        p.start()
        procs.append(p)

    origen, coord, n = q.get()

    for p in procs:
        p.join()

    t1 = time.perf_counter()
    return {"start": origen, "coord": coord, "iter": n}, (t1 - t0)

ganador, tiempo = concurso_procesos(starts)
print("ganador:", ganador)
print("tiempo concurso (procesos):", tiempo)


1) ¿Hay mejora en rendimiento al usar hilos? (Las Lupas con threads)
En general, para este problema (CPU-bound) no debería haber una mejora real al usar hilos en CPython por culpa del GIL (Global Interpreter Lock), que impide que varios hilos ejecuten bytecode de Python simultáneamente en varios núcleos. Si en alguna ejecución observas que “parece” ir más rápido, suele deberse a la aleatoriedad del paseo (algunas búsquedas encuentran antes un dorado por suerte) y a efectos de planificación/overhead, pero no a paralelismo efectivo; por eso, la conclusión correcta es que los hilos no escalan bien en tareas intensivas de CPU en Python estándar.

2) Concurso (8 lupas a la vez, parar cuando una gana): ¿tarda más o menos que una sola lupa y por qué?
De media, el concurso con 8 lupas tarda menos que una sola lupa porque estás haciendo 8 búsquedas independientes y te quedas con la primera que acierta: estadísticamente reduces el tiempo de espera al mínimo de varias variables aleatorias (más “boletos” a la vez). En otras palabras, aumentas la probabilidad de que alguna encuentre un hash dorado pronto, así que el tiempo esperado baja (aunque el número total de intentos combinados pueda ser similar o incluso mayor).

3) Procesos para “Las Lupas” (8 procesos para encontrar 8 dorados): ¿mejora el rendimiento? ¿motivo?
Sí, con procesos sí es razonable esperar mejora en un problema CPU-bound, porque cada proceso tiene su propio intérprete y su propio GIL, permitiendo paralelismo real en distintos núcleos del CPU. A diferencia de los hilos, aquí el trabajo se reparte de verdad entre cores, y por eso el tiempo total suele disminuir (hasta el límite marcado por el número de núcleos, el coste de crear procesos y la comunicación mínima).

4) Concurso con procesos: ¿otra forma de comunicación entre procesos? ¿sería más eficiente?
Además de Event + Queue, se puede comunicar con Pipe (rápido y simple, especialmente 1 a 1), con Value/Array (memoria compartida para banderas/contadores, muy eficiente para datos simples), con un Manager (más cómodo pero normalmente más lento por usar proxies), o con shared_memory (muy eficiente para datos grandes, pero más complejo). Para este caso (solo “avisar ganador y devolver una tupla”), Event + Queue o Event + Pipe suele ser de las opciones más eficientes y limpias; Manager es fácil, pero suele penalizar rendimiento.